In [19]:
!pip install torch-geometric

In [20]:
import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv, GATConv, GINConv
from torch_geometric.datasets import CoraFull, DeezerEurope
from torch_geometric.utils import train_test_split_edges, negative_sampling, subgraph
from torch_geometric.utils import to_dense_adj, to_undirected, train_test_split_edges
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

corafull = CoraFull(root='CoraFull')[0]
deezerEurope = DeezerEurope(root='DeezerEurope')[0]

datasets = {'CoraFull': corafull.to(device), 'DeezerEurope': deezerEurope.to(device)}

In [21]:
print (corafull)
print (deezerEurope)

Data(x=[19793, 8710], edge_index=[2, 126842], y=[19793])
Data(x=[28281, 128], edge_index=[2, 185504], y=[28281])


In [22]:
from sklearn.metrics import pairwise_distances
import numpy as np

def compute_nodes_in_triangle(edge_index, num_nodes):
    """
    Identify nodes in Q: nodes part of at least one triangle in the graph.
    """
    adj = to_dense_adj(edge_index, max_num_nodes=num_nodes).squeeze(0)
    triangles = torch.mm(adj, adj.T) * adj  # Triangle adjacency
    Q = torch.nonzero(torch.sum(triangles, dim=1)).squeeze()
    return Q[:1000]

In [23]:
# Split Datasets into train, test and validation
train_ratio, val_ratio, test_ratio = 0.6, 0.2, 0.2

num_nodes = {}
edge_index = {}

train_nodes = {}
train_data = {}
train_edge_index = {}

test_nodes = {}
test_data = {}
test_edge_index = {}
test_neg_edge_index = {}
Q = {}

val_nodes = {}
val_data = {}
val_edge_index = {}
val_neg_edge_index = {}

for dataset_name in datasets:
    num_nodes[dataset_name] = datasets[dataset_name].num_nodes
    edge_index[dataset_name] = datasets[dataset_name].edge_index

    edge_splits = train_test_split_edges(datasets[dataset_name], test_ratio=test_ratio, val_ratio=val_ratio)
    
    train_edge_index[dataset_name] = to_undirected(edge_splits.train_pos_edge_index)
    test_edge_index[dataset_name] = to_undirected(edge_splits.test_pos_edge_index)
    val_edge_index[dataset_name] = to_undirected(edge_splits.val_pos_edge_index)

    train_nodes[dataset_name] = torch.sort(torch.unique(train_edge_index[dataset_name]))[0].to(device)
    train_data[dataset_name] = datasets[dataset_name].x[train_nodes[dataset_name]].to(device)
    train_edge_index[dataset_name] = subgraph(train_nodes[dataset_name], train_edge_index[dataset_name], relabel_nodes=True)[0].to(device)    


    test_nodes[dataset_name] = torch.sort(torch.unique(test_edge_index[dataset_name]))[0].to(device)
    test_data[dataset_name] = datasets[dataset_name].x[test_nodes[dataset_name]].to(device)
    test_edge_index[dataset_name] = subgraph(test_nodes[dataset_name], test_edge_index[dataset_name], relabel_nodes=True)[0].to(device)

    test_neg_edge_index[dataset_name] = negative_sampling(edge_index=test_edge_index[dataset_name],
                                                          num_nodes=test_nodes[dataset_name].shape[0],
                                                          num_neg_samples=test_edge_index[dataset_name].shape[1]).to(device)

    Q[dataset_name] = compute_nodes_in_triangle(test_edge_index[dataset_name], test_nodes[dataset_name].shape[0])
    
    
    val_nodes[dataset_name] = torch.sort(torch.unique(val_edge_index[dataset_name]))[0].to(device)
    val_data[dataset_name] = datasets[dataset_name].x[val_nodes[dataset_name]].to(device)
    val_edge_index[dataset_name] = subgraph(val_nodes[dataset_name], val_edge_index[dataset_name], relabel_nodes=True)[0].to(device)

    val_neg_edge_index[dataset_name] = negative_sampling(edge_index=val_edge_index[dataset_name], 
                                                         num_nodes=val_nodes[dataset_name].shape[0],
                                                         num_neg_samples=val_edge_index[dataset_name].shape[1]).to(device)

In [24]:
print (train_edge_index['CoraFull'].shape)
print (test_edge_index['CoraFull'].shape)
print (val_edge_index['CoraFull'].shape)

print (train_edge_index['DeezerEurope'].shape)
print (test_edge_index['DeezerEurope'].shape)
print (val_edge_index['DeezerEurope'].shape)

torch.Size([2, 76106])
torch.Size([2, 25368])
torch.Size([2, 25368])
torch.Size([2, 111304])
torch.Size([2, 37100])
torch.Size([2, 37100])


In [25]:
# GNN Model Definition
class GNNModel(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels=64, out_channels=64, gnn_type="GCN"):
        super(GNNModel, self).__init__()
        if gnn_type == "GCN":
            self.conv1 = GCNConv(in_channels, hidden_channels)
            self.conv2 = GCNConv(hidden_channels, out_channels)
        elif gnn_type == "GAT":
            self.conv1 = GATConv(in_channels, hidden_channels)
            self.conv2 = GATConv(hidden_channels, out_channels)
        elif gnn_type == "GIN":
            self.conv1 = GINConv(torch.nn.Sequential(
                torch.nn.Linear(in_channels, hidden_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(hidden_channels, hidden_channels)
            ))
            self.conv2 = GINConv(torch.nn.Sequential(
                torch.nn.Linear(hidden_channels, out_channels),
                torch.nn.ReLU(),
                torch.nn.Linear(out_channels, out_channels)
            ))
        else:
            raise ValueError("Unsupported GNN type")

    def forward(self, input, edge_index):
        hidden_output = F.relu(self.conv1(input, edge_index))
        output = self.conv2(hidden_output, edge_index)
        return output

In [26]:
def auc_loss(node_embeddings, pos_edge_index, neg_edge_index, margin=1.0):
    # Positive edge scores
    pos_u, pos_v = pos_edge_index
    pos_scores = F.cosine_similarity(node_embeddings[pos_u], node_embeddings[pos_v])

    # Negative edge scores
    neg_u, neg_v = neg_edge_index
    neg_scores = F.cosine_similarity(node_embeddings[neg_u], node_embeddings[neg_v])
    
    # Margin-based ranking loss
    loss = F.relu(margin + neg_scores - pos_scores).mean()
    return loss

In [27]:
from rich.console import Console
from rich.table import Table

console = Console()

def train(model, optimizer, train_data, train_edge_index, val_data, val_edge_index, val_neg_edge_index, epochs=200, val_interval=20):
    model.to(device)
    model.train()

    for epoch in range(1, epochs + 1):
        train_neg_edge_index = negative_sampling(edge_index=train_edge_index, 
                                                 num_nodes=train_data.shape[0], 
                                                 num_neg_samples=train_edge_index.shape[1]).to(device)

        node_embeddings = model(train_data, train_edge_index)
        loss = auc_loss(node_embeddings, train_edge_index, train_neg_edge_index)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if epoch % val_interval == 0:
            model.eval()
            with torch.no_grad():
                node_embeddings = model(val_data, val_edge_index)
                val_loss = auc_loss(node_embeddings, val_edge_index, val_neg_edge_index)
            model.train()

            console.print(f"Epoch {epoch:3d} | [cyan]Train Loss:[/] {loss:.5f} | [green]Val Loss:[/] {val_loss:.5f}")

In [28]:
models = {}

for dataset_name in datasets:
    models[dataset_name] = {'GCN': None, 'GAT': None, 'GIN': None}
    
    for gnn_model in models[dataset_name]:
        console.rule(f"[bold blue]Dataset: {dataset_name}, GNN Model: {gnn_model}")
        
        model = GNNModel(datasets[dataset_name].num_features, gnn_type=gnn_model)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)
    
        train(model, optimizer, train_data[dataset_name], train_edge_index[dataset_name], 
              val_data[dataset_name], val_edge_index[dataset_name], val_neg_edge_index[dataset_name])

        models[dataset_name][gnn_model] = model

──────────────────────────────────────── Dataset: CoraFull, GNN Model: GCN ────────────────────────────────────────

Epoch  20 | Train Loss: 0.22831 | Val Loss: 0.21772

Epoch  40 | Train Loss: 0.17888 | Val Loss: 0.17305

Epoch  60 | Train Loss: 0.15238 | Val Loss: 0.14511

Epoch  80 | Train Loss: 0.13419 | Val Loss: 0.12993

...

Epoch 120 | Train Loss: 0.23890 | Val Loss: 0.22298

Epoch 140 | Train Loss: 0.23283 | Val Loss: 0.22326

Epoch 160 | Train Loss: 0.23047 | Val Loss: 0.22224

Epoch 180 | Train Loss: 0.22450 | Val Loss: 0.22620

Epoch 200 | Train Loss: 0.22232 | Val Loss: 0.22271

In [29]:
from collections import defaultdict
import numpy as np

def generate_random_hyperplanes(num_features, num_planes):
    return torch.randn(num_features, num_planes).to(device)

def projection_hash(features, hyperplanes):
    projections = torch.mm(features, hyperplanes)
    hash_codes = (projections > 0).float()
    return hash_codes

def get_hash_codes(train_features, test_features, num_planes=8):
    hyperplanes = generate_random_hyperplanes(train_features.shape[1], num_planes)    
    train_hash_codes = projection_hash(train_features, hyperplanes)

    powers_of_two = torch.pow(2, torch.arange(num_planes - 1, -1, -1)).float().to(device)
    
    train_hash_codes_dict = defaultdict(list)
    for i, train_hash_code in enumerate(train_hash_codes):
        key = torch.dot(train_hash_code, powers_of_two).int().item()
        train_hash_codes_dict[key].append(i)

    for code in train_hash_codes_dict.keys():
        train_hash_codes_dict[code] = torch.tensor(train_hash_codes_dict[code]).to(device)
    
    test_hash_codes = projection_hash(test_features, hyperplanes)
    test_hash_codes = [torch.dot(test_hash_code, powers_of_two).item() for test_hash_code in test_hash_codes]
    
    return train_hash_codes_dict, test_hash_codes

In [30]:
def get_evaluation(scores, labels, K_values):
    ranked_nodes = torch.argsort(scores, descending=True)
    rankings = torch.argsort(ranked_nodes)

    precision = {}
    for k in K_values:
        precision[k] = (labels[ranked_nodes[:k]].sum() / k).item()
    mrr = 0
    if torch.sum(labels) != 0:
        rank_of_first_positive = torch.min(rankings[torch.nonzero(labels)]).item() + 1
        mrr = 1 / rank_of_first_positive

    return precision, mrr

In [31]:
import numpy as np

def evaluate_inference(model, test_data, test_edge_index, Q, K_values):
    model.eval()
    with torch.no_grad():
        precision_scores = {k: [] for k in K_values}
        mrr_scores = []
        adj = to_dense_adj(test_edge_index, max_num_nodes=test_data.shape[0]).squeeze(0)
        
        embeddings = model(test_data, test_edge_index)

        embeddings_hash_codes_dict, Q_hash_codes = get_hash_codes(embeddings, embeddings[Q])
        
        for q, q_hash_code in zip(Q, Q_hash_codes):
            subset_indices = embeddings_hash_codes_dict[q_hash_code]
            scores = F.cosine_similarity(embeddings[q].unsqueeze(0), embeddings[subset_indices], dim=1)
            scores[torch.nonzero(subset_indices == q, as_tuple=True)[0].item()] = float('-inf')
            labels = adj[q][subset_indices]
    
            p, m = get_evaluation(scores, labels, K_values)
            for k in K_values:
                precision_scores[k].append(p[k])
            mrr_scores.append(m)
        
        # Average metrics
        precision_avg = {k: np.mean(precision_scores[k]) for k in K_values}
        mrr_avg = np.mean(mrr_scores)
        
    return precision_avg, mrr_avg

In [32]:
import time

K_values = [1, 5, 10]

for dataset_name in datasets:
    console.rule(f"[bold blue]Dataset: {dataset_name}")

    table = Table(show_header=True, header_style="bold green")
    table.add_column("GNN Models", style="dim", justify="center")
    for k in K_values:
        table.add_column(f"Precision@{k}", style="bold magenta", justify="center")
    table.add_column("MRR", style="bold magenta", justify="center")
    table.add_column("Inference Time (s)", style="bold red", justify="center")

    for gnn_model in models[dataset_name]:
        start_time = time.time()
        model = models[dataset_name][gnn_model]
        precision_avg, mrr_avg = evaluate_inference(model, test_data[dataset_name], test_edge_index[dataset_name], Q[dataset_name], K_values)
        end_time = time.time()
        
        row = [gnn_model]
        for k in K_values:
            row.append(f"{precision_avg[k]:.5f}")
        row.append(f"{mrr_avg:.5f}")
        row.append(f"{(end_time - start_time):.5f}")
        table.add_row(*row)
    
    console.print(table)

──────────────────────────────────────────────── Dataset: CoraFull ────────────────────────────────────────────────

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ GNN Models ┃ Precision@1 ┃ Precision@5 ┃ Precision@10 ┃   MRR   ┃ Inference Time (s) ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│    GCN     │   0.57816   │   0.29724   │   0.19218    │ 0.67598 │      1.20730       │
│    GAT     │   0.67126   │   0.33494   │   0.20759    │ 0.74510 │      1.19064       │
│    GIN     │   0.47471   │   0.25333   │   0.17034    │ 0.58311 │      1.22110       │
└────────────┴─────────────┴─────────────┴──────────────┴─────────┴────────────────────┘

────────────────────────────────────────────── Dataset: DeezerEurope ──────────────────────────────────────────────

┏━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ GNN Models ┃ Precision@1 ┃ Precision@5 ┃ Precision@10 ┃   MRR   ┃ Inference Time (s) ┃
┡━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│    GCN     │   0.52566   │   0.21552   │   0.12691    │ 0.58541 │      1.33188       │
│    GAT     │   0.53191   │   0.22253   │   0.13054    │ 0.58098 │      1.35425       │
│    GIN     │   0.28786   │   0.14168   │   0.09812    │ 0.38773 │      1.34480       │
└────────────┴─────────────┴─────────────┴──────────────┴─────────┴────────────────────┘